# **ETL**

#### Realizaremos el proceso de Extracción, Transformación y Carga (`ETL`) de los datos necesarios para el proyecto, obteniéndolos de varias fuentes confiables.

---

In [1]:
# Librerias a utilizar
import requests
import pandas as pd
import calendar
import io
import datetime
import json
import re
from selenium import webdriver
from bs4 import BeautifulSoup

---

### *Datos mundiales de terremotos de 1970 a 2023*

In [25]:
# Extraemos los datos mediante la API de la USGS

# Creamos lista donde almacenaremos todos los datos
all_data = []

# Iteramos para extraer los datos de los años de interes, en este caso de 1970 a 2023 filtrando los mayores a magnitud 4
for i in range(1970, 2023):
    year = i
    for mes in range (1, 13):
        # Tomamos el último dia de cada mes
        last_day = calendar.monthrange(year, mes)[1]
        starttime = f"{year}-{mes}-01"
        endtime = f"{year}-{mes}-{last_day}"
        # El url de la api con los parámetros, donde filtramos buscando magnitudes mayores a 4
        url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={starttime}&endtime={endtime}&minmagnitude=4"
        # Envía la solicitud HTTP a la API de USGS y procesa los datos de respuesta
        response = requests.get(url)
        data = json.loads(response.text)
        data = data["features"]
        # Itera sobre cada objeto "Feature" en la lista "data"
        for feature in data:
            # Accede a la parte "properties" de cada objeto que es donde se encuentran los datos de interés
            properties = feature['properties']
            properties['year'] = year
            properties['month'] = mes
            # Agregamos los datos a la lista
            all_data.append(properties)
   
    
# Realizamos la misma extracción, pero para el año 2023 de forma actualizada siempre al tiempo actual
# Tomamos el mes actual
mes_actual = datetime.datetime.now().month
# Iteramos por mes, con tope en el mes actual
for mes in range (1, mes_actual +1):
    # Tomamos el último dia de cada mes
    last_day = calendar.monthrange(2023, mes)[1]
    starttime = f"2023-{mes}-01"
    endtime = f"2023-{mes}-{last_day}"
    url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={starttime}&endtime={endtime}&minmagnitude=4"
    response = requests.get(url)
    data = json.loads(response.text)
    data = data["features"]
    # Itera sobre cada objeto "Feature" en la lista "data"
    for feature in data:
        # Accede a la parte "properties" de cada objeto
        properties = feature['properties']
        properties['year'] = 2023
        properties['month'] = mes
        # Crea un dataframe de Pandas a partir de la lista de eventos sísmicos
        all_data.append(properties)
            
# Concatenamos todos los dataframes en uno solo
df_global = pd.DataFrame(all_data)

# El código tarda mucho en ejecutarse, habria que optimizarlo. Pero posiblemente la causa es que se esta iterando por año y luego por mes. 
# Por lo que serían muchas iteraciones.

In [26]:
# Observamos
df_global

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,types,nst,dmin,rms,gap,magType,type,title,year,month
0,4.09,"65km ENE of Beatty, NV",2566800720,1454030943020,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",13.0,1.749,0.31,253.0,ml,nuclear explosion,"M 4.1 Nuclear Explosion - 65km ENE of Beatty, NV",1970,1
1,5.90,"59 km NNE of Port-Olry, Vanuatu",2536102930,1652124015110,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,3.894,...,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,"M 5.9 - 59 km NNE of Port-Olry, Vanuatu",1970,1
2,6.14,Fiji region,2415962630,1651009857198,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,M 6.1 - Fiji region,1970,1
3,5.27,"195 km WSW of Linxia Chengguanzhen, China",2285370330,1651009855849,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,"M 5.3 - 195 km WSW of Linxia Chengguanzhen, China",1970,1
4,5.77,"38 km SSE of San Cristóbal, Venezuela",2280584330,1651009854455,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,6.276,...,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,"M 5.8 - 38 km SSE of San Cristóbal, Venezuela",1970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461814,5.30,"82 km ESE of Katsuren-haebaru, Japan",1682907747800,1682917939040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,2.0,2.7,NaN,...,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,"M 5.3 - 82 km ESE of Katsuren-haebaru, Japan",2023,5
461815,5.10,"Ryukyu Islands, Japan",1682907660774,1682909229040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",77.0,0.888,1.17,64.0,mb,earthquake,"M 5.1 - Ryukyu Islands, Japan",2023,5
461816,4.30,"45 km NNE of Klyuchi, Russia",1682905534116,1682906478040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",40.0,3.949,0.63,126.0,mb,earthquake,"M 4.3 - 45 km NNE of Klyuchi, Russia",2023,5
461817,5.10,None,1682900181305,1682902782040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",66.0,0.682,0.59,121.0,mww,earthquake,M 5.1 -,2023,5


In [27]:
# Observamos más en detalle, se pueden evidenciar columnas con muchos valores faltantes
df_global.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461819 entries, 0 to 461818
Data columns (total 28 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   mag      461819 non-null  float64
 1   place    460422 non-null  object 
 2   time     461819 non-null  int64  
 3   updated  461819 non-null  int64  
 4   tz       2 non-null       float64
 5   url      461819 non-null  object 
 6   detail   461819 non-null  object 
 7   felt     38764 non-null   float64
 8   cdi      38764 non-null   float64
 9   mmi      18487 non-null   float64
 10  alert    7568 non-null    object 
 11  status   461819 non-null  object 
 12  tsunami  461819 non-null  int64  
 13  sig      461819 non-null  int64  
 14  net      461819 non-null  object 
 15  code     461819 non-null  object 
 16  ids      461819 non-null  object 
 17  sources  461819 non-null  object 
 18  types    461819 non-null  object 
 19  nst      176315 non-null  float64
 20  dmin     140706 non-null  

In [28]:
# Vemos mas en detalle
df_global.isna().sum().sort_values(ascending=False).head(10)

tz       461817
alert    454251
mmi      443332
cdi      423055
felt     423055
dmin     321113
nst      285504
gap      188194
rms       55407
place      1397
dtype: int64

In [29]:
# Observamos las columnas, hay algunas que no aportan nada de información útil para el análisis o tienen muchos nulos
df_global.columns

Index(['mag', 'place', 'time', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi',
       'mmi', 'alert', 'status', 'tsunami', 'sig', 'net', 'code', 'ids',
       'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'type',
       'title', 'year', 'month'],
      dtype='object')

In [30]:
# Eliminamos las columnas que no utilizaremos
df_global.drop(['time', 'updated',"url","detail","ids","sources","title","status","tz","alert"],inplace=True,axis=1)

In [31]:
# Observamos que no todos los datos son de terremotos
df_global["type"].value_counts()

earthquake           461048
nuclear explosion       627
volcanic eruption        62
explosion                33
mine collapse            18
quarry blast              9
rock burst                7
mining explosion          7
other event               4
sonic boom                2
landslide                 1
collapse                  1
Name: type, dtype: int64

In [32]:
# Filtramos solo los "earthquake" y reiniciamos el índice
df_global = df_global[df_global["type"] == "earthquake"].reset_index()

In [33]:
# Observamos
df_global

,index,mag,place,felt,cdi,mmi,tsunami,sig,net,code,types,nst,dmin,rms,gap,magType,type,year,month
0,1,5.90,"59 km NNE of Port-Olry, Vanuatu",NaN,NaN,3.894,0,536,iscgemsup,800507,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
1,2,6.14,Fiji region,NaN,NaN,NaN,0,580,iscgem,800471,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
2,3,5.27,"195 km WSW of Linxia Chengguanzhen, China",NaN,NaN,NaN,0,427,iscgem,800431,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
3,4,5.77,"38 km SSE of San Cristóbal, Venezuela",NaN,NaN,6.276,0,512,iscgem,800429,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
4,5,5.72,"20 km ESE of Lata, Solomon Islands",NaN,NaN,NaN,0,503,iscgem,800427,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461043,461814,5.30,"82 km ESE of Katsuren-haebaru, Japan",2.0,2.7,NaN,0,433,us,7000jxaa,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,2023,5
461044,461815,5.10,"Ryukyu Islands, Japan",NaN,NaN,NaN,0,400,us,7000jxab,",origin,phase-data,",77.0,0.888,1.17,64.0,mb,earthquake,2023,5
461045,461816,4.30,"45 km NNE of Klyuchi, Russia",NaN,NaN,NaN,0,284,us,7000jxa6,",origin,phase-data,",40.0,3.949,0.63,126.0,mb,earthquake,2023,5
461046,461817,5.10,None,NaN,NaN,NaN,0,400,us,7000jx9j,",origin,phase-data,",66.0,0.682,0.59,121.0,mww,earthquake,2023,5


In [34]:
# Exportamos
df_global.to_parquet("Datasets/Usa_Jap_Mex/terremotos_global.parquet",compression="snappy")

---

# Creamos tabla de países

In [35]:
# Creamos un diccionario con los países y los ID de cada uno para la normalización de la base de datos
country = { "idCountry" : [1, 2, 3] ,
             "country" : ["United States", "Japón", "México"]   }
# Creamos un dataframe con estos datos
df_country = pd.DataFrame(country)
# Observamos
df_country

,idCountry,country
0,1,United States
1,2,Japón
2,3,México


In [36]:
# Exportamos
df_country.to_json("Datasets/Usa_Jap_Mex/países_id.json")

---

# USA

In [37]:
# Hacemos una lista de los estados
estados2 = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawai', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
       'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming']


In [38]:
# Observamos si hay nulos, ya que estos no nos permitiran filtrar
df_global["place"].isna().sum()

1397

In [39]:
# Rellenamos con "sin dato"
df_global["place"].fillna("sin dato",inplace=True)

In [40]:
# Filtramos en el dataframe global con la lista de estados de Estados Unidos
all_dfs2 = []
for i in estados2 : 
    all_dfs2.append(df_global[df_global["place"].str.contains(i)])

# Luego concatenamos 
df_usa1 = pd.concat(all_dfs2, ignore_index=True)   

In [41]:
# Observamos
df_usa1

,index,mag,place,felt,cdi,mmi,tsunami,sig,net,code,types,nst,dmin,rms,gap,magType,type,year,month
0,11488,4.50,"3 km NNW of Fayette, Alabama",NaN,NaN,5.447,0,312,us,p0000brg,",origin,shakemap,trump-shakemap,",NaN,NaN,NaN,NaN,mb,earthquake,1975,6
1,138172,4.80,"15 km NNW of Flomaton, Alabama",NaN,NaN,6.012,0,354,us,p00089hq,",impact-text,origin,phase-data,shakemap,trump-...",NaN,NaN,0.98,NaN,mb,earthquake,1997,10
2,186431,4.60,"8 km S of Valley Head, Alabama",17204.0,6.0,4.987,0,926,se,605341,",associate,dyfi,focal-mechanism,impact-text,lo...",21.0,NaN,0.07,154.0,mw,earthquake,2003,4
3,204159,4.30,"4 km W of Forkland, Alabama",85.0,5.0,NaN,0,327,us,p000d7vw,",associate,dyfi,impact-text,moment-tensor,orig...",61.0,NaN,0.90,78.9,mwr,earthquake,2004,11
4,9,5.73,"Rat Islands, Aleutian Islands, Alaska",NaN,NaN,NaN,0,505,iscgem,800290,",impact-text,origin,trump-origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22351,320005,4.80,"20 km W of Fort Washakie, Wyoming",116.0,4.3,3.570,0,404,us,b000jx4l,",dyfi,losspager,moment-tensor,origin,phase-dat...",NaN,0.37800,1.08,16.0,mwr,earthquake,2013,9
22352,326935,4.83,"22 km S of Mammoth, Wyoming",22.0,2.9,5.570,0,700,uu,60061837,",cap,dyfi,general-link,general-text,losspager,...",35.0,0.04763,0.25,63.0,mw,earthquake,2014,3
22353,357568,4.00,"23 km SE of Kelly, Wyoming",153.0,3.4,4.370,0,298,us,10004t1f,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.19500,1.16,18.0,mwr,earthquake,2016,2
22354,364635,4.80,"4 km ENE of Bondurant, Wyoming",578.0,3.9,4.640,0,580,us,20006umx,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.37300,1.14,7.0,mwr,earthquake,2016,8


In [42]:
# Separamos la columna place para tener la distancia y el estado en distintas columnas
df_usa1["place"] = df_usa1["place"].str.strip()
df_usa1[["distance","location","location2"]] = df_usa1["place"].str.split(",",expand=True)
# Borramos la columna location2 y place ya que no nos sirven
df_usa1.drop(["place","location2","type"],inplace=True,axis=1)
# Agregamos el ID correspondiente a USA que es 1
df_usa1["idCountry"] = 1
# Observamos
df_usa1

,index,mag,felt,cdi,mmi,tsunami,sig,net,code,types,nst,dmin,rms,gap,magType,year,month,distance,location,idCountry
0,11488,4.50,NaN,NaN,5.447,0,312,us,p0000brg,",origin,shakemap,trump-shakemap,",NaN,NaN,NaN,NaN,mb,1975,6,3 km NNW of Fayette,Alabama,1
1,138172,4.80,NaN,NaN,6.012,0,354,us,p00089hq,",impact-text,origin,phase-data,shakemap,trump-...",NaN,NaN,0.98,NaN,mb,1997,10,15 km NNW of Flomaton,Alabama,1
2,186431,4.60,17204.0,6.0,4.987,0,926,se,605341,",associate,dyfi,focal-mechanism,impact-text,lo...",21.0,NaN,0.07,154.0,mw,2003,4,8 km S of Valley Head,Alabama,1
3,204159,4.30,85.0,5.0,NaN,0,327,us,p000d7vw,",associate,dyfi,impact-text,moment-tensor,orig...",61.0,NaN,0.90,78.9,mwr,2004,11,4 km W of Forkland,Alabama,1
4,9,5.73,NaN,NaN,NaN,0,505,iscgem,800290,",impact-text,origin,trump-origin,",NaN,NaN,NaN,NaN,mw,1970,1,Rat Islands,Aleutian Islands,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22351,320005,4.80,116.0,4.3,3.570,0,404,us,b000jx4l,",dyfi,losspager,moment-tensor,origin,phase-dat...",NaN,0.37800,1.08,16.0,mwr,2013,9,20 km W of Fort Washakie,Wyoming,1
22352,326935,4.83,22.0,2.9,5.570,0,700,uu,60061837,",cap,dyfi,general-link,general-text,losspager,...",35.0,0.04763,0.25,63.0,mw,2014,3,22 km S of Mammoth,Wyoming,1
22353,357568,4.00,153.0,3.4,4.370,0,298,us,10004t1f,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.19500,1.16,18.0,mwr,2016,2,23 km SE of Kelly,Wyoming,1
22354,364635,4.80,578.0,3.9,4.640,0,580,us,20006umx,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.37300,1.14,7.0,mwr,2016,8,4 km ENE of Bondurant,Wyoming,1


In [43]:
# Vemos las columnas
df_usa1.columns

Index(['index', 'mag', 'felt', 'cdi', 'mmi', 'tsunami', 'sig', 'net', 'code',
       'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'year', 'month',
       'distance', 'location', 'idCountry'],
      dtype='object')

In [44]:
# Reordenamos para mejor entendimiento
df_usa1 = df_usa1[['idCountry','year','month','location','distance','types', 'code','mag', 'felt', 'cdi', 'mmi', 'tsunami', 'sig', 'net', 'nst', 'dmin', 'rms', 'gap', 'magType']]

# Observamos
df_usa1.sample(5)

,idCountry,year,month,location,distance,types,code,mag,felt,cdi,mmi,tsunami,sig,net,nst,dmin,rms,gap,magType
22256,1,2008,9,None,off the coast of Washington,",focal-mechanism,impact-text,moment-tensor,ori...",p000ggwm,5.2,NaN,NaN,NaN,0,416,us,103.0,NaN,NaN,147.0,mwc
4063,1,1988,12,Alaska,61 km ESE of Adak,",origin,phase-data,",p0003q6k,4.5,NaN,NaN,NaN,0,312,us,NaN,NaN,1.30,NaN,mb
1847,1,1981,10,Alaska,21 km NNE of Huslia,",origin,phase-data,",p0001gjq,4.2,NaN,NaN,NaN,0,271,us,NaN,NaN,0.60,NaN,ml
12546,1,2014,8,Alaska,144 km ESE of Attu Station,",associate,cap,origin,phase-data,",b000s3m7,4.4,NaN,NaN,NaN,0,298,us,NaN,0.739,0.53,107.0,mb
7800,1,2001,8,Alaska,85 km ESE of Egegik,",origin,phase-data,",001b2mcx8k,4.0,NaN,NaN,NaN,0,246,ak,NaN,NaN,0.57,NaN,mb


In [45]:
# Exportamos
df_usa1.to_json("Datasets/Usa_Jap_Mex/Usa.json")

---

# Japón

In [46]:
# Filtramos el dataframe global con los datos de Japón
df_japon = df_global[df_global["place"].str.contains("Japan")]
# Observamos
df_japon

,index,mag,place,felt,cdi,mmi,tsunami,sig,net,code,types,nst,dmin,rms,gap,magType,type,year,month
11,12,6.40,"45 km SSW of Obihiro, Japan",NaN,NaN,6.419,0,630,iscgem,800254,",origin,shakemap,trump-shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
74,77,5.53,"Volcano Islands, Japan region",NaN,NaN,NaN,0,470,iscgem,799066,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,2
101,107,5.54,"67 km NE of Naze, Japan",NaN,NaN,NaN,0,472,iscgem,798599,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
119,127,5.70,"101 km WSW of Kurio, Japan",NaN,NaN,NaN,0,500,iscgem,798320,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
121,129,5.70,"4 km WSW of Takanosu, Japan",NaN,NaN,3.951,0,500,iscgem,798303,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461039,461810,5.10,"75 km ESE of Katsuren-haebaru, Japan",NaN,NaN,NaN,0,400,us,7000jxal,",origin,phase-data,",58.0,0.883,0.99,79.0,mww,earthquake,2023,5
461040,461811,5.00,"81 km SE of Taira, Japan",NaN,NaN,NaN,0,385,us,7000jxaj,",origin,phase-data,",44.0,0.845,0.95,78.0,mb,earthquake,2023,5
461041,461812,5.90,"79 km ESE of Katsuren-haebaru, Japan",10.0,3.1,3.841,0,539,us,7000jxag,",dyfi,internal-moment-tensor,internal-origin,l...",76.0,0.871,0.75,57.0,mww,earthquake,2023,5
461043,461814,5.30,"82 km ESE of Katsuren-haebaru, Japan",2.0,2.7,NaN,0,433,us,7000jxaa,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,2023,5


In [ ]:
# Agregamos el ID de japon que es 2
df_japon["idCountry"] = 2

In [48]:
# Reordenamos 
df_japon = df_japon[['idCountry','year','month','place','types', 'code','mag', 'felt', 'cdi', 'mmi', 'tsunami', 'sig', 'net', 'nst', 'dmin', 'rms', 'gap', 'magType']]

# Observamos
df_japon.sample(5)

,idCountry,year,month,place,types,code,mag,felt,cdi,mmi,tsunami,sig,net,nst,dmin,rms,gap,magType
392321,2,2018,9,"78 km ESE of Yamada, Japan",",origin,phase-data,",1000h5rx,4.4,NaN,NaN,NaN,0,298,us,NaN,2.044,0.60,157.0,mb
459445,2,2023,3,"50 km ESE of Shizunai-furukawach?, Japan",",dyfi,origin,phase-data,",7000jiya,4.8,3.0,2.7,NaN,0,355,us,149.0,0.229,0.75,80.0,mb
456660,2,2023,1,"100 km ENE of Miyako, Japan",",origin,phase-data,",7000j6dh,4.5,NaN,NaN,NaN,0,312,us,22.0,1.797,0.89,217.0,mb
14402,2,1976,2,"36 km SE of Nemuro, Japan",",origin,shakemap,",p0000ey3,5.6,NaN,NaN,4.162,0,482,us,NaN,NaN,NaN,NaN,mb
107388,2,1994,3,"125 km W of Kamiiso, Japan",",impact-text,origin,phase-data,",p00069av,4.3,NaN,NaN,NaN,0,284,us,NaN,NaN,1.20,NaN,mb


In [49]:
# Exportamos
df_japon.to_json("Datasets/Usa_Jap_Mex/Japón.json")

---

# México

In [50]:
# A través del servicio sismológico nacional de México se extraen los siguientes datos a través de una URL
url = "http://www2.ssn.unam.mx:8080/catalogo/csv/20230509150323SUMPD.csv"   # La url es posible que se caiga ya que solo se encuentra activa un par de horas
                                                                            # en todo caso se puede generar otra de aca http://www2.ssn.unam.mx:8080/catalogo/
# Hacemos una solicitud a la url obteniendo el contenido
response = requests.get(url).content

# StringIO se utiliza para convertir el contenido en una cadena legible por pandas
df_mexico = pd.read_csv(io.StringIO(response.decode('utf-8')), skiprows=4, skipfooter=7, engine='python') # con skiprows salteamos las primeras y últimas filas que generan errores

# Observamos
df_mexico

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,1970-02-03,23:08:50,6.6,15.524,-99.493,21.0,"141 km al SUR de SAN MARCOS, GRO",1970-02-04,05:08:50,revisado
1,1970-04-29,08:01:34,7.3,14.463,-92.683,44.0,"62 km al SUROESTE de CD HIDALGO, CHIS",1970-04-29,14:01:34,revisado
2,1970-04-30,02:32:58,6.5,14.608,-93.260,22.0,"100 km al SUROESTE de MAPASTEPEC, CHIS",1970-04-30,08:32:58,revisado
3,1971-09-30,02:18:00,6.5,26.880,-110.800,14.0,"92 km al SUROESTE de PUEBLO YAQUI, SON",1971-09-30,08:18:00,revisado
4,1972-10-20,02:17:46,6.6,18.700,-106.756,10.0,"238 km al OESTE de CIHUATLAN, JAL",1972-10-20,08:17:46,revisado
...,...,...,...,...,...,...,...,...,...,...
42697,2023-05-08,20:30:27,4.0,16.490,-95.140,2.1,"9 km al SUROESTE de CD IXTEPEC, OAX",2023-05-09,02:30:27,verificado
42698,2023-05-08,22:24:25,4.0,16.080,-95.080,12.2,"17 km al SURESTE de SALINA CRUZ, OAX",2023-05-09,04:24:25,verificado
42699,2023-05-09,00:16:55,4.2,16.110,-97.500,8.9,"13 km al NOROESTE de RIO GRANDE, OAX",2023-05-09,06:16:55,verificado
42700,2023-05-09,03:56:09,4.6,16.340,-95.310,13.9,"8 km al OESTE de TEHUANTEPEC, OAX",2023-05-09,09:56:09,verificado


In [51]:
# Separamos "referencia de localizacion" en "Distancia" y "Estado"
df_mexico[["Distancia","Estado"]] = df_mexico["Referencia de localizacion"].str.split(",",expand=True)
# Agregamos el ID de México que es 3
df_mexico["idCountry"] = 3
# Observamos
df_mexico.head()

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus,Distancia,Estado,idCountry
0,1970-02-03,23:08:50,6.6,15.524,-99.493,21.0,"141 km al SUR de SAN MARCOS, GRO",1970-02-04,05:08:50,revisado,141 km al SUR de SAN MARCOS,GRO,3
1,1970-04-29,08:01:34,7.3,14.463,-92.683,44.0,"62 km al SUROESTE de CD HIDALGO, CHIS",1970-04-29,14:01:34,revisado,62 km al SUROESTE de CD HIDALGO,CHIS,3
2,1970-04-30,02:32:58,6.5,14.608,-93.260,22.0,"100 km al SUROESTE de MAPASTEPEC, CHIS",1970-04-30,08:32:58,revisado,100 km al SUROESTE de MAPASTEPEC,CHIS,3
3,1971-09-30,02:18:00,6.5,26.880,-110.800,14.0,"92 km al SUROESTE de PUEBLO YAQUI, SON",1971-09-30,08:18:00,revisado,92 km al SUROESTE de PUEBLO YAQUI,SON,3
4,1972-10-20,02:17:46,6.6,18.700,-106.756,10.0,"238 km al OESTE de CIHUATLAN, JAL",1972-10-20,08:17:46,revisado,238 km al OESTE de CIHUATLAN,JAL,3


In [52]:
# Eliminamos las columnas que no serán de utilidad para el análisis
df_mexico.drop(["Referencia de localizacion","Fecha UTC","Hora UTC","Estatus"],axis=1,inplace=True)
# Observamos
df_mexico

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Distancia,Estado,idCountry
0,1970-02-03,23:08:50,6.6,15.524,-99.493,21.0,141 km al SUR de SAN MARCOS,GRO,3
1,1970-04-29,08:01:34,7.3,14.463,-92.683,44.0,62 km al SUROESTE de CD HIDALGO,CHIS,3
2,1970-04-30,02:32:58,6.5,14.608,-93.260,22.0,100 km al SUROESTE de MAPASTEPEC,CHIS,3
3,1971-09-30,02:18:00,6.5,26.880,-110.800,14.0,92 km al SUROESTE de PUEBLO YAQUI,SON,3
4,1972-10-20,02:17:46,6.6,18.700,-106.756,10.0,238 km al OESTE de CIHUATLAN,JAL,3
...,...,...,...,...,...,...,...,...,...
42697,2023-05-08,20:30:27,4.0,16.490,-95.140,2.1,9 km al SUROESTE de CD IXTEPEC,OAX,3
42698,2023-05-08,22:24:25,4.0,16.080,-95.080,12.2,17 km al SURESTE de SALINA CRUZ,OAX,3
42699,2023-05-09,00:16:55,4.2,16.110,-97.500,8.9,13 km al NOROESTE de RIO GRANDE,OAX,3
42700,2023-05-09,03:56:09,4.6,16.340,-95.310,13.9,8 km al OESTE de TEHUANTEPEC,OAX,3


In [53]:
# Observamos las columnas
df_mexico.columns

Index(['Fecha', 'Hora', 'Magnitud', 'Latitud', 'Longitud', 'Profundidad',
       'Distancia', 'Estado', 'idCountry'],
      dtype='object')

In [54]:
# Reordenamos
df_mexico = df_mexico[['idCountry','Fecha', 'Hora','Distancia', 'Estado', 'Latitud', 'Longitud','Magnitud', 'Profundidad']]
# Observamos
df_mexico

,idCountry,Fecha,Hora,Distancia,Estado,Latitud,Longitud,Magnitud,Profundidad
0,3,1970-02-03,23:08:50,141 km al SUR de SAN MARCOS,GRO,15.524,-99.493,6.6,21.0
1,3,1970-04-29,08:01:34,62 km al SUROESTE de CD HIDALGO,CHIS,14.463,-92.683,7.3,44.0
2,3,1970-04-30,02:32:58,100 km al SUROESTE de MAPASTEPEC,CHIS,14.608,-93.260,6.5,22.0
3,3,1971-09-30,02:18:00,92 km al SUROESTE de PUEBLO YAQUI,SON,26.880,-110.800,6.5,14.0
4,3,1972-10-20,02:17:46,238 km al OESTE de CIHUATLAN,JAL,18.700,-106.756,6.6,10.0
...,...,...,...,...,...,...,...,...,...
42697,3,2023-05-08,20:30:27,9 km al SUROESTE de CD IXTEPEC,OAX,16.490,-95.140,4.0,2.1
42698,3,2023-05-08,22:24:25,17 km al SURESTE de SALINA CRUZ,OAX,16.080,-95.080,4.0,12.2
42699,3,2023-05-09,00:16:55,13 km al NOROESTE de RIO GRANDE,OAX,16.110,-97.500,4.2,8.9
42700,3,2023-05-09,03:56:09,8 km al OESTE de TEHUANTEPEC,OAX,16.340,-95.310,4.6,13.9


In [55]:
# Exportamos a formato json
df_mexico.to_json("Datasets/Usa_Jap_Mex/México.json")

---

# Requerimientos de Data Analysts

---

+ ### KPI 1 

In [56]:
# Se utilizan los datasets de la carpeta KPI 1

In [11]:
# Leemos el archivo json
df_polizas = pd.read_json("Datasets/KPI 1/polizas_japon.json")
# Agregamos el ID
df_polizas["idCountry"] = 2
# Observamos
df_polizas

,nombre Estado,Propiedades aseguradas,Monto asegurado,Primas emitidas,Propiedades Aseguradas,Monto asegurado.1,Tipo Construcción,año,idCountry
0,Hokkaido,105197,1370980,2253888,222997,2853620,Tipo A edificios,2019,2
1,Aomori,11104,154461,237145,22744,313867,Tipo A edificios,2019,2
2,Iwate,12400,180140,277996,31708,444239,Tipo A edificios,2019,2
3,Miyagi,57775,854537,1998684,151576,2133825,Tipo A edificios,2019,2
4,Akita,7314,98260,139790,16368,218498,Tipo A edificios,2019,2
...,...,...,...,...,...,...,...,...,...
559,Kumamoto,40154,99993,363675,124323,314686,Tipo B residencias,2021,2
560,Oita,11281,25408,155190,34069,79374,Tipo B residencias,2021,2
561,Miyazaki,14739,32067,198889,44211,103315,Tipo B residencias,2021,2
562,Kagoshima,22104,44871,169119,68987,148932,Tipo B residencias,2021,2


In [2]:
# Ingestamos el archivo de excel, que contiene 7 hojas de utilidad
seguros_japon_terremotos = pd.read_excel("Datasets\KPI 1\SegurosJaponTerremotos.xlsx",sheet_name=None)

# Almacenamos las hojas en un diccionario de Pandas
hojas = {}
for nombre_hoja, datos_hoja in seguros_japon_terremotos.items():
    hojas[nombre_hoja] = datos_hoja

# Accedemos cada hoja
datos_hoja11 = hojas["nuevas polizas"]
datos_hoja22= hojas["monto asegurado"]
datos_hoja33 = hojas["seguro contra terremotos"]
datos_hoja44 = hojas["descuentos seguros"]
datos_hoja55 = hojas[" tasa penetración hogares"]

# Agregamos el ID = 2 ya que los datos corresponden a japon
datos_hoja11["idCountry"] = 2
datos_hoja22["idCountry"] = 2
datos_hoja33["idCountry"] = 2
datos_hoja44["idCountry"] = 2
datos_hoja55["idCountry"] = 2

# Los exportamos 
datos_hoja11.to_json("Datasets/KPI 1/nuevas polizas.json")
datos_hoja22.to_json("Datasets/KPI 1/monto asegurado.json")
datos_hoja33.to_json("Datasets/KPI 1/seguro contra terremotos.json")
datos_hoja44.to_json("Datasets/KPI 1/descuentos seguros.json")
datos_hoja55.to_json("Datasets/KPI 1/tasa penetración hogares.json")

# Y a continuación observaremos cada uno de ellos...

In [5]:
datos_hoja11

,Año,Numero de polizas en vigencia,Suma Asegurada,Primas de Polizas pagadas en miles de yenes,idCountry
0,2015,9501454,79535689,245480878,2
1,2016,9298612,77478280,254208714,2
2,2017,8941885,74571308,261314984,2
3,2018,9308162,79432981,297305589,2
4,2019,9511036,82522929,318344114,2
5,2020,9559989,82882120,336313105,2
6,2021,9015086,77648023,340879821,2


In [6]:
datos_hoja22

,Año Fiscal,Cantidad de polizas emitidas,Valor Total de montos asegurados,idCountry
0,2015,16941425,150272904,2
1,2016,17712801,159628458,2
2,2017,18257927,167054068,2
3,2018,19005841,176604126,2
4,2019,19740800,186105521,2
5,2020,20355462,195021847,2
6,2021,20804068,202415533,2


In [7]:
datos_hoja33

,Prefectura,Cantidad de contratos,Monto asegurado,Prima del seguro,Cantidad de Polizas en vigor,Monto asegurado.1,Año,idCountry
0,Hokkaido,356092,3082650,7030077,806318,7363120,2021,2
1,Aomori,66987,540895,1279328,143072,1237956,2021,2
2,Iwate,61203,587165,1378451,142609,1454489,2021,2
3,Miyagi,245963,2147149,7758799,543630,5317508,2021,2
4,Akita,43396,395870,1027096,106751,1022449,2021,2
...,...,...,...,...,...,...,...,...
136,Kumamoto,160468,1442015,3076633,339564,3131920,2019,2
137,Oita,63326,573389,1934760,150265,1432046,2019,2
138,Miyazaki,66160,550069,1821805,150103,1320455,2019,2
139,Kagoshima,107018,893959,1865712,240056,2088168,2019,2


In [8]:
datos_hoja44

,Tipo de descuento,Cantidad de polizas nuevas,Monto Asegurado nuevo en millones de Yen,Primas de seguros nuevo en miles de Yenes,Cantidad de polizas vigentes,Cantidad total de seguros contratados en millones de Yen,Año,idCountry
0,Descuento de Edificio con aislamiento Sismico,26515,274526,762169,76457,803165,2021,2
1,Descuento de Edificio con resistencia sismica ...,462712,6940240,20987222,1303605,19508456,2021,2
2,Descuento de Edificio con resistencia sismica ...,64089,802050,3027975,164273,2167456,2021,2
3,Descuento de Edificio con resistencia sismica ...,102801,832532,4079211,288257,2328570,2021,2
4,Descuento de En diagnostico de resistencia sis...,10717,95570,666481,28796,298282,2021,2
5,Descuento de Año de construcción de la propiedad,5522625,51991775,232490602,12884890,135591413,2021,2
6,Sin Descuento (sin reducción de precio),2825627,16711330,78866162,6057790,41718191,2021,2
7,Descuento de Edificio con aislamiento Sismico,28413,295098,769327,70968,731817,2020,2
8,Descuento de Edificio con resistencia sismica ...,427647,6270175,17692529,1152676,17111322,2020,2
9,Descuento de Edificio con resistencia sismica ...,59790,760162,2623604,156798,2048033,2020,2


In [9]:
datos_hoja55

,Prefecture,2015,2016,2017,2018,2019,2020,2021,2019 año Calendario Hogar,2020 año Calendario Hogar,2021 año Calendario Hogar,idCountry
0,Hokkaido,51.0,52.4,53.3,56.6,59.1,60.6,61.8,26.7,27.7,28.6,2
1,Aomori,61.8,62.9,63.9,65.5,67.0,68.6,70.1,22.5,23.1,23.9,2
2,Iwate,66.8,67.9,69.1,70.4,72.3,73.7,75.0,25.3,25.9,26.6,2
3,Miyagi,86.2,86.4,86.3,86.8,87.0,87.5,88.7,52.0,51.9,52.7,2
4,Akita,68.5,69.5,70.8,72.0,73.3,74.4,74.8,23.5,24.2,24.9,2
5,Yamagata,60.9,62.5,63.3,64.8,66.3,67.9,68.7,23.7,24.4,25.4,2
6,Fukushima,70.5,72.2,73.1,74.1,75.2,76.7,79.3,31.8,32.7,34.4,2
7,Ibaraki,60.5,61.9,62.2,63.8,64.6,66.0,66.5,30.4,31.2,31.8,2
8,Tochigi,62.2,64.2,65.6,67.6,69.7,71.4,72.7,30.5,31.7,32.7,2
9,Gumma,54.7,56.6,57.6,59.9,62.2,63.9,65.0,24.7,25.8,26.8,2


In [12]:
# Cargamos
polizas_vigencia = pd.read_json("Datasets/KPI 1/polizas_en_vigencia.json")
# Añadimos el ID 
polizas_vigencia["idCountry"] = 2
# Observamos
polizas_vigencia

,Año,Numero de polizas en vigencia,Suma Asegurada,Primas de Polizas pagadas en miles de yenes,idCountry
0,2015,9501454,79535689,245480878,2
1,2016,9298612,77478280,254208714,2
2,2017,8941885,74571308,261314984,2
3,2018,9308162,79432981,297305589,2
4,2019,9511036,82522929,318344114,2
5,2020,9559989,82882120,336313105,2
6,2021,9015086,77648023,340879821,2


In [9]:
# Leemos el archivo parquet
seg_mex = pd.read_parquet("Datasets/KPI 1/seguros_mex.parquet")
# Agregamos el ID
seg_mex["idCountry"] = 3
# Observamos
seg_mex

,AñO,MONEDA,ZONA SISMICA,TIPO DE SEGURO,SUBTIPO DE SEGURO,NUMERO DE PISOS,NUMERO DE UBICACIONES,PRIMA EMITIDA,PRIMA RETENIDA,PRIMA DEVENGADA,COMISION DIRECTA,VALORES TOTALES EDIFICIO,VALORES TOTALES CONTENIDOS,VALORES TOTALES PeERDIDAS CONSECUENCIALES,LIMITE MAXIMO DE RESPONSABILIDAD,idCountry
0,2019,Nacional,A_,Terremoto,Otro,1,1,0,0,63,0,0,655000,0,655000,3
1,2019,Nacional,B_,Terremoto,Otro,2,2,383,262,382,19,0,1500000,120000,1620000,3
2,2019,Nacional,C_,Terremoto,Otro,1,3,3511,2336,2970,175,5000000,4000000,0,9000000,3
3,2019,Nacional,C_,Terremoto,Otro,2,9,2895,1790,2883,146,2859280,8600000,1150000,12609280,3
4,2019,Nacional,B_,Terremoto,Otro,1,17,6135,331,79334,305,890890000,245448050,81172500,1217510550,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321787,2021,Extranjera,B_,Terremoto,Microseguro,1,1,34337,343,10724,1030,0,306600000,29199991,335799991,3
321788,2021,Extranjera,C_,Terremoto,Microseguro,1,1,4,0,2,0,0,40880,4088,44968,3
321789,2021,Extranjera,B_,Terremoto,Microseguro,1,2,7463,-101,4110,224,0,83804000,8380400,92184400,3
321790,2021,Extranjera,B1,Terremoto,Microseguro,1,4,5809,267,3199,174,0,59547368,7998737,67546105,3


In [12]:
# Exportamos los 4 archivos
df_polizas.to_json("Datasets/KPI 1/polizas_japon.json")
polizas_vigencia.to_json("Datasets/KPI 1/polizas_en_vigencia.json")
seg_mex.to_parquet("Datasets/KPI 1/seguros_mex.parquet")

---

+ ### KPI 2

In [62]:
# USA : https://www.worlddata.info/america/usa/earthquakes.php

# Japon : https://www.worlddata.info/asia/japan/earthquakes.php

# Mexico : https://www.worlddata.info/america/mexico/earthquakes.php

In [63]:
# Creamos una variable con los URL para realizar el Web Scrapping
urls = ["https://www.worlddata.info/america/mexico/earthquakes.php", # mexico
        "https://www.worlddata.info/asia/japan/earthquakes.php", # japon
        "https://www.worlddata.info/america/usa/earthquakes.php"] # usa

# Creamos una variable para colocar el ID de cada pais
idCountry = 4

# Creamos una lista donde guardaremos todos los datos
data_total = []

# Iteramos en cada URL 
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    seccion = soup.find('table', {'class': 'std100 hover'})
    rows = seccion.find_all("tr")
    idCountry -= 1
    # Hacemos el web scrapping
    for row in rows[1:]:
        cols = row.find_all('td')
        date = cols[0].text.strip()
        location = cols[1].text.strip()
        depth = cols[2].text.strip()
        magnitude = cols[3].text.strip()
        deaths = cols[4].text.strip()
        idCountry = idCountry
        # Agregamos todo en una lista
        data_total.append([idCountry, date, location, depth, magnitude, deaths])

# Creamos un dataframe con esos datos
df_fatalidades = pd.DataFrame(data_total, columns=["idCountry","date", "location", "depth", "magnitude", "deaths"])  

# Colocamos el tipo de dato correcto en Date
df_fatalidades['date'] = pd.to_datetime(df_fatalidades['date'], format='%m/%d/%Y')

# Observamos
df_fatalidades    

,idCountry,date,location,depth,magnitude,deaths
0,3,2022-09-22,"Mexico City, Michoacan",24 km,6.8,3
1,3,2022-09-19,"Michoacan, Colima, Jalisco",15 km,7.6,2
2,3,2022-05-25,Oaxaca,34 km,5.5,0
3,3,2021-09-08,Guerrero,20 km,7.0,3
4,3,2020-06-23,Oaxaca,26 km,7.4,10
...,...,...,...,...,...,...
350,1,1954-08-23,Stillwater Range,,6.8,0
351,1,1954-07-06,Fallon,,6.8,0
352,1,1952-08-22,Kern County,24 km,5.8,2
353,1,1952-07-21,Kern County,14 km,7.7,12


In [64]:
# Exportamos
df_fatalidades.to_json("Datasets/KPI 2/Fatalidades.json")

---

+ ### KPI 3

In [65]:
# Extraeremos la información a través de web scrapping para los datos de USA y Japon

# Creamos lista donde estarán los dfs finales
dfs_totales = []

# Iteramos por año ya que necesitamos iterar en la página de 2015 a 2022
for year in range(2015, 2023):
    website = f'https://www.shakeout.org/glb_participants.php?year={year}&start=All'
    path = 'driver/chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(website)
    filas = driver.find_elements_by_tag_name('tr')
    datos = []
    # Extraemos la informacion
    for fila in filas:
        datos.append(fila.text)
    iteraciones = datos[32:]
    datos_paises = []
    # La separamos y normalizamos
    for i in range(len(iteraciones)):
        estado, conteo = iteraciones[i].split(" \n")
        datos_paises.append(estado.strip())
        datos_paises.append(conteo.strip().replace(",", ""))
        datos_paises.append(year)

    # Creamos dos listas vacías para almacenar los nombres y los números
    names = []
    numbers = []
    years = []

    # Iteramos sobre la lista de datos y agregamos los elementos a las listas correspondientes
    for i, element in enumerate(datos_paises):
        if i % 3 == 0:
            names.append(element)
        elif i % 3 == 1:
            numbers.append(element)
        else:
            years.append(element)

    # Creamos un diccionario con las listas de datos
    dict_data = {'pais': names, 'participantes': numbers, 'año': years}

    # Creamos el dataframe
    df = pd.DataFrame(dict_data)
    # Lo agregamos a la lista
    dfs_totales.append(df)
    
    # Cerramos el navegador
    driver.quit()    

# Unificamos todos los dataframes de la lista
df_participantes_usa_jap = pd.concat(dfs_totales, ignore_index=True)

# Colocamos el ID en 1 
df_participantes_usa_jap["idCountry"] = 1

# Donde es Japon el ID será 2
df_participantes_usa_jap.loc[df_participantes_usa_jap['pais'] == 'Japan', 'idCountry'] = 2

# Luego a la lista de estados de USA, le agregamos Japon
estados2.append("Japan")

# Filtramos en el dataframe global con esa lista agregandolos en "all_dfs7"
all_dfs7 = []
for i in estados2 : 
    all_dfs7.append(df_participantes_usa_jap[df_participantes_usa_jap["pais"].str.contains(i)])

# Luego concatenamos el resultado final
df_participantes_usa_jap = pd.concat(all_dfs7, ignore_index=True)   

# Observamos
df_participantes_usa_jap

,pais,participantes,año,idCountry
0,Alaska,126981,2015,1
1,Alaska,126833,2016,1
2,Alaska,126395,2017,1
3,Alaska,140632,2018,1
4,Alaska,176560,2019,1
...,...,...,...,...
115,Japan,6297584,2018,2
116,Japan,6837776,2019,2
117,Japan,3328410,2020,2
118,Japan,2131481,2021,2


In [66]:
# Comprobamos
df_participantes_usa_jap[df_participantes_usa_jap["pais"] == "Washington State"]

,pais,participantes,año,idCountry
96,Washington State,1084030,2015,1
97,Washington State,1101578,2016,1
98,Washington State,1264502,2017,1
99,Washington State,1385627,2018,1
100,Washington State,1545326,2019,1
101,Washington State,976158,2020,1
102,Washington State,1362232,2021,1
103,Washington State,1390163,2022,1


In [67]:
# Realizaremos otro Web Scrapping, pero esta vez buscando los datos de México
lista_mexico = []
# Iteramos a través del año
for year in range (2015,2023) :     
    website = f'https://www.shakeout.org/participants.php?year={year}'
    path = 'driver/chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(website)
    filas = driver.find_elements_by_tag_name('tr')
    datos = []
    # Iteramos separando y normalizando los datos
    for fila in filas:
        datos.append(fila.text + " " + str(year))
    # Filtramos a Mexico    
    datos_filtrados = [dato for dato in datos if "Mexico" in dato]    
    # Separamos los datos
    pais, participantes, año = datos_filtrados[0].split(" ")
    # Agregamos a la lista final
    lista_mexico.append([pais, participantes, año])

    # Cerramos el navegador
    driver.quit()
 
# Concatenamos el resultado final 
df_participantes_mexico = pd.DataFrame(lista_mexico,columns=["pais","participantes","año"])
# Agregamos el ID = 3
df_participantes_mexico["idCountry"] = 3
# Observamos
df_participantes_mexico

,pais,participantes,año,idCountry
0,Mexico,"732,982",2015,3
1,Mexico,"732,982",2016,3
2,Mexico,"7,735,476",2017,3
3,Mexico,"8,731,167",2018,3
4,Mexico,"9,730,616",2019,3
5,Mexico,297,2020,3
6,Mexico,"5,071,926",2021,3
7,Mexico,"6,003,647",2022,3


In [68]:
# Realizamos el último Web Scrapping del KPI 3, esta vez buscando datos de las categorias
dfs = []
# También iteramos por año
for year in range (2015,2024):
    # Cargamos la página con Selenium
    url = f"https://www.shakeout.org/glb_participants.php?year={year}&start=All"
    path = 'driver/chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(url)
    driver.implicitly_wait(10)

    # Ejecutamos el código JavaScript en la página
    script = """
    return [
        createCatNewRow("Individuals/Families", "individuals"),
        createCatNewRow("Childcare and Pre-Schools", "childcare"),
        createCatNewRow("K-12 Schools and Districts", "schools"),
        createCatNewRow("Colleges and Universities", "colleges"),
        createCatNewRow("Local Government", "local"),
        createCatNewRow("State Government", "state"),
        createCatNewRow("Federal/National Government (Including Military)", "federal"),
        createCatNewRow("Tribes/Indigenous Peoples", "tribes"),
        createCatNewRow("Businesses", "businesses"),
        createCatNewRow("Hotels and Other Lodgings", "lodging"),
        createCatNewRow("Healthcare", "healthcare"),
        createCatNewRow("Senior Facilities/Communities", "seniors"),
        createCatNewRow("Disability/AFN Organizations", "disability"),
        createCatNewRow("Non-Profit Organizations", "nonprofits"),
        createCatNewRow("Neighborhood Groups", "neighborhoods"),
        createCatNewRow("Preparedness Organizations", "preparedness"),
        createCatNewRow("Faith-based Organizations", "faithbased"),
        createCatNewRow("Museums, Libraries, Parks, etc.", "museums"),
        createCatNewRow("Volunteer/Service Clubs", "service"),
        createCatNewRow("Youth Organizations", "youth"),
        createCatNewRow("Animal Shelter/Service Providers", "animalservices"),
        createCatNewRow("Agriculture/Livestock Sector", "agriculture"),
        createCatNewRow("Volunteer Radio Groups", "radiogroups"),
        createCatNewRow("Science/Engineering Organizations", "science"),
        createCatNewRow("Media Organizations", "media"),
        createCatNewRow("Other", "other")
    ];
    """
    # Obtenemos el resultado en una lista
    categories = driver.execute_script(script)
    # Renombramos
    rows = categories
    # Utilizamos re para filtrar los datos de nuestro interés 
    pattern = re.compile(r'<tr><td align="left">(.*?)</td><td align="right" style="font-size:14px;">(.*?)</td></tr>')
    data = []
    # Iteramos aplicando el filtro
    for row in rows:
        match = pattern.findall(row)
        if len(match) > 0:
            categoria = match[0][0]
            valor = int(match[0][1].replace(',', ''))
            data.append({'Categoria': categoria, 'Valor': valor, "Año" : year})

    # Creamos un dataframe con los datos de ese año        
    df = pd.DataFrame(data)
    # Lo agregamos a la lista final
    dfs.append(df)
    # Cerramos el navegador
    driver.quit()

# Concatenamos el resultado final
df_simulacros = pd.concat(dfs)
# Agregamos su ID
df_simulacros["idSimulacro"] = 1
# Observamos
df_simulacros

,Categoria,Valor,Año,idSimulacro
0,Individuals/Families,40127,2015,1
1,Childcare and Pre-Schools,174239,2015,1
2,K-12 Schools and Districts,29033389,2015,1
3,Colleges and Universities,3622510,2015,1
4,Local Government,907190,2015,1
...,...,...,...,...
21,Agriculture/Livestock Sector,9,2023,1
22,Volunteer Radio Groups,874,2023,1
23,Science/Engineering Organizations,1513,2023,1
24,Media Organizations,272,2023,1


In [69]:
# Unimos los participantes de mexico con los de usa y japon
df_participantes_simulacros = pd.concat([df_participantes_usa_jap,df_participantes_mexico],axis=0)
# Agregamos el ID de la tabla anterior
df_participantes_simulacros["idSimulacro"] = 1
# Observamos
df_participantes_simulacros

,pais,participantes,año,idCountry,idSimulacro
0,Alaska,126981,2015,1,1
1,Alaska,126833,2016,1,1
2,Alaska,126395,2017,1,1
3,Alaska,140632,2018,1,1
4,Alaska,176560,2019,1,1
...,...,...,...,...,...
3,Mexico,"8,731,167",2018,3,1
4,Mexico,"9,730,616",2019,3,1
5,Mexico,297,2020,3,1
6,Mexico,"5,071,926",2021,3,1


In [70]:
# Reseteamos los índices para poder exportarlo en formato JSON
df_participantes_simulacros = df_participantes_simulacros.reset_index(drop=True)
df_simulacros = df_simulacros.reset_index(drop=True)

In [71]:
# Exportamos los 2 archivos
df_participantes_simulacros.to_json("Datasets/KPI 3/Participantes_simulacros_paises.json")
df_simulacros.to_json("Datasets/KPI 3/Participantes_simulacros_categorias.json")

---

+ ### KPI 4 

In [72]:
# Se utiliza el archivo Excel de la carpeta Datasets/KPI 4.

In [91]:
# Ingestamos el archivo de excel, que contiene 7 hojas de utilidad
df_seg_vida = pd.read_excel("Datasets\KPI 4\Seguros VidaPersonasJapon2016-2021.xlsx",sheet_name=None)

# Almacenamos las hojas en un diccionario de Pandas
hojas = {}
for nombre_hoja, datos_hoja in df_seg_vida.items():
    hojas[nombre_hoja] = datos_hoja

# Accedemos cada hoja
datos_hoja1 = hojas["1"]
datos_hoja2 = hojas["2"]
datos_hoja3 = hojas["3"]
datos_hoja4 = hojas["4"]
datos_hoja5 = hojas["5"]
datos_hoja6 = hojas["6"]
datos_hoja7 = hojas["7"]


# Agregamos el ID = 2 ya que los datos corresponden a japon
datos_hoja1["idCountry"] = 2
datos_hoja2["idCountry"] = 2
datos_hoja3["idCountry"] = 2
datos_hoja4["idCountry"] = 2
datos_hoja5["idCountry"] = 2
datos_hoja6["idCountry"] = 2
datos_hoja7["idCountry"] = 2

# Y a continuación observaremos cada uno de ellos...

In [100]:
datos_hoja1.sample(3)

,Año fiscal,Tipo de Seguro,Cantidad de Polizas,Numero de personas nuevas aseguradas para accidentes personales,Valor de la prima en miles de Yen,Numero de reclamos pagados,Reclamos pagados en miles de Yen,idCountry
20,2019,Seguro de Accidentes Personales de Tráfico,1584006,6606878,12335302,27170,4591693,2
18,2019,Seguro Ordinario de Accidentes Personales,3611769,23860470,179384079,669998,75886516,2
32,2021,Seguro de Accidentes Personales de Tráfico,1529360,5705939,10827923,21944,3477256,2


In [101]:
datos_hoja2.sample(3)

,Clase de ocupación,No de Polizas,Muerte o discapacidad posterior al accidente No de Asegurados,Monto asegurado en Million Yen,Prima o cuota del seguro en miles de yen,No de Asegurados,Monto diario de seguro en Miles de Yen,Primas Escritas Thousand Yen,No de Asegurados.1,Cantidad de asegurados por dia Thousand Yen,primas escritas Thousand Yen,Total primas escritas Thousand Yen,Ano,idCountry
8,Clase A,842385,6977077,30820823,21028140,5488442,22086737,10777894,4873422,10743596,20558965,52365001,2019,2
0,Clase A,422559,5720506,26234887,17356297,4411789,18283304,8678603,3817817,8549665,16292792,42327693,2021,2
6,Contrato aplicado al promedio ponderado,2834240,13046439,53177143,47763845,9925113,49673703,38621208,6141322,14242152,30514097,116899150,2020,2


In [102]:
datos_hoja3.sample(3)

,Clase de ocupación,Cantidad Pagos de reclamaciones por muerte,Reclamos pagados por muerte,Cantidad Pagos de reclamos por inhabilidad permanente,Reclamos pagados por Inabilidad Permanente,Cantidad Reclamos por Hospitalización,Pagos reclamaciones por hospitalización,Cantidad de reclamaciones pagadas por cirugia,Pagos por cirugia reclamados,Cantidad de reclamaciones por visita ambulatoria,Pagos de reclamaciones por visita ambulatoria,Cantidad total de reclamaciones,Pagos totales pagados en reclamaciones,Ano,idCountry
0,Clase A,568,2608280,3590,4430708,24228,2995234,15382,702344,133878,7389476,177646,18126045,2021,2
8,Clase A,689,3179148,4354,5975296,28130,3704797,17708,829510,167499,9382933,218380,23071686,2019,2
1,Clase B,57,439551,521,991732,1855,346385,1490,82434,9986,824778,13909,2684882,2021,2


In [103]:
datos_hoja4.sample(3)

,Clase Ocupacional Nuevos Contratos,Descuento por cobertura de accidentes laborales,No de Polizas,Muerte / Incapacidad Permanente No de Asegurados,Muerte / Incapacidad Permanente Monto Asegurado Million Yen,Muerte / Incapacidad Permanente Prima Escrita Thousand Yen,Hospitalizacion No de Asegurados,Hospitalizacion Monto Asegurado Por Dia Thousand Yen,Hospitalizacion Prima Escrita Thousand Yen,Visita Ambulatoria No de Asegurados,Visita Ambulatoria Monto Asegurado Por Dia Thousand Yen,Visita Ambulatoria Prima Escrita Thousand Yen,Total Prima Escrita Thousand Yen,Año,idCountry
12,Clase A,Aplicado,70416,1855378,11768090,3671692,1552985,6306896,1331238,1052475,3086672,2627325,7630256,2020,2
27,Clase B,Aplicado,44071,247187,2148048,1681278,221142,1241940,667783,208876,682057,1358849,3707911,2019,2
30,prima media,Aplicado,53285,761203,6146827,2620207,575573,2848847,919679,515213,1503301,1803431,5343318,2019,2


In [104]:
datos_hoja5.sample(3)

,Clase Ocupacional Reclamos,Descuento por cobertura de accidentes laborales,No de Reclamos Pagados por Muerte,Reclamos pagados Por Muerte Thousand Yen,Incapacidad Permanente No de Reclamos Pagados,Incapacidad Permanente Reclamos Pagados Thousand Yen,Hospitalizacion No de Reclamos Pagados,Hospitalizacion Reclamos Pagados Thousand Yen,Cirugia No de Reclamos Pagados,Cirugia Reclamos Pagados Thousand Yen,Visita ambulatoria No de Reclamos Pagados,Visita ambulatoria Reclamos Pagados Thousand Yen,Total No de Reclamos Pagados,Total Reclamos Pagados Thousand Yen,Año,idCountry
28,Class B,No Aplica,38,142825,344,524711,1386,227289,984,47981,7927,680810,10679,1623618,2019,2
10,Total,No Aplica,3186,9632520,13388,13131266,97623,13291284,64176,3487129,390820,21036162,569193,60578363,2021,2
24,Class A,Aplica,60,494076,659,1335929,2518,417729,1865,91883,14171,923519,19273,3263138,2019,2


In [105]:
datos_hoja6.sample(3)

,Clase Ocupacional Nuevos Contratos,No de Polizas,Muerte / Incapacidad Permanente No de Asegurados,Muerte / Incapacidad Permanente Monto Asegurado Million Yen,Muerte / Incapacidad Permanente Prima Escrita Thousand Yen,Hospitalizacion No de Asegurados,Hospitalizacion Monto Asegurado Por Dia Thousand Yen,Hospitalizacion Prima Escrita Thousand Yen,Visita Ambulatoria No de Asegurados,Visita Ambulatoria Monto Asegurado Por Dia Thousand Yen,Visita Ambulatoria Prima Escrita Thousand Yen,Total Prima Escrita Thousand Yen,Año,idCountry
13,Esposo,-297.101,1858475,5175805,3676827,1537651,6813734,3248677,1437852,2819167,6015281,12940786,2020,2
23,Total,324452,2067678,16590622,15289538,1758147,20556828,13733462,1635747,8138209,20680793,49703795,2019,2
21,Esposo,-279.57,1996212,5756434,4077856,1683320,7864845,3609118,1574718,2977529,6307296,13994271,2019,2


In [106]:
datos_hoja7.sample(3)

,Clase Ocupacional Reclamos,No de Reclamos Pagados por Muerte,Reclamos pagados Por Muerte Thousand Yen,Incapacidad Permanente No de Reclamos Pagados,Incapacidad Permanente Reclamos Pagados Thousand Yen,Hospitalizacion No de Reclamos Pagados,Hospitalizacion Reclamos Pagados Thousand Yen,Cirugia No de Reclamos Pagados,Cirugia Reclamos Pagados Thousand Yen,Visita ambulatoria No de Reclamos Pagados,Visita ambulatoria Reclamos Pagados Thousand Yen,Total No de Reclamos Pagados,Total Reclamos Pagados Thousand Yen,año,idCountry
0,Clase A,96,293727,563,648307,3287,442740,2075,113919,21491,1164662,27512,2663357,2021,2
4,Subtotal,388,1057010,2484,2307507,11151,1639478,8210,436470,87896,4359008,110129,9799476,2021,2
14,Otros Parientes,215,502123,1785,984231,11024,1638913,7455,316788,91563,2501593,112042,5943649,2020,2


---

+ ### KPI 5

In [73]:
# https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=US-JP-MX

In [74]:
# Ingestaremos los datos a través de una URL
url = "https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=excel"
df_gdp = pd.read_excel(url)
# Observamos, se puede evidenciar que los nombres de las columnas estan mal posicionados
df_gdp.head()

,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,Last Updated Date,2023-03-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Country Name,Country Code,Indicator Name,Indicator Code,1.960000e+03,1.961000e+03,1.962000e+03,1.963000e+03,1.964000e+03,1.965000e+03,...,2.012000e+03,2.013000e+03,2.014000e+03,2.015000e+03,2.016000e+03,2.017000e+03,2.018000e+03,2.019000e+03,2.020000e+03,2.021000e+03
3,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.615084e+09,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.368970e+09,2.610039e+09,3.126019e+09
4,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.129152e+10,2.180944e+10,2.370806e+10,2.821128e+10,2.611994e+10,2.968348e+10,...,9.720022e+11,9.826771e+11,1.003403e+12,9.231439e+11,8.898593e+11,1.030482e+12,1.016697e+12,1.009052e+12,9.341791e+11,1.089454e+12


In [75]:
# Corregimos las columnas
df_gdp.columns = df_gdp.iloc[2]
# Eliminamos las filas
df_gdp = df_gdp.drop([0,1,2]) 
# Reiniciamos el índice
df_gdp = df_gdp.reset_index()
# Como podemos observar nos da datos de todos los países y solo precisamos Usa, Japón y México para este análisis
df_gdp.head(3)

2,index,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
0,3,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,...,2.615084e+09,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.368970e+09,2.610039e+09,3.126019e+09
1,4,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.129152e+10,2.180944e+10,2.370806e+10,2.821128e+10,2.611994e+10,...,9.720022e+11,9.826771e+11,1.003403e+12,9.231439e+11,8.898593e+11,1.030482e+12,1.016697e+12,1.009052e+12,9.341791e+11,1.089454e+12
2,5,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,...,2.020357e+10,2.056449e+10,2.055058e+10,1.999816e+10,1.801956e+10,1.889635e+10,1.841885e+10,1.890449e+10,2.014344e+10,1.478686e+10


In [76]:
# Filtramos los países de interés para nuestro análisis
df_gdp = df_gdp.loc[df_gdp["Country Name"].isin(["United States", "Japan", "Mexico"])].reset_index()
# Observamos
df_gdp

2,level_0,index,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
0,119,122,Japan,JPN,GDP (current US$),NY.GDP.MKTP.CD,4.430734e+10,5.350862e+10,6.072302e+10,6.949813e+10,...,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12,5.003678e+12,4.930837e+12,5.037835e+12,5.123318e+12,5.040108e+12,4.940878e+12
1,154,157,Mexico,MEX,GDP (current US$),NY.GDP.MKTP.CD,1.304000e+10,1.416000e+10,1.520000e+10,1.696000e+10,...,1.201090e+12,1.274443e+12,1.315351e+12,1.171868e+12,1.078491e+12,1.158913e+12,1.222408e+12,1.269012e+12,1.090515e+12,1.272839e+12
2,251,254,United States,USA,GDP (current US$),NY.GDP.MKTP.CD,5.433000e+11,5.633000e+11,6.051000e+11,6.386000e+11,...,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13,1.869511e+13,1.947734e+13,2.053306e+13,2.138098e+13,2.106047e+13,2.331508e+13


In [77]:
# Eliminamos las columnas que no son de utilidad
df_gdp.drop(["level_0","index","Indicator Code","Indicator Name"],axis=1,inplace=True)
# Observamos
df_gdp

2,Country Name,Country Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,...,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0
0,Japan,JPN,4.430734e+10,5.350862e+10,6.072302e+10,6.949813e+10,8.174901e+10,9.095028e+10,1.056281e+11,1.237819e+11,...,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12,5.003678e+12,4.930837e+12,5.037835e+12,5.123318e+12,5.040108e+12,4.940878e+12
1,Mexico,MEX,1.304000e+10,1.416000e+10,1.520000e+10,1.696000e+10,2.008000e+10,2.184000e+10,2.432000e+10,2.656000e+10,...,1.201090e+12,1.274443e+12,1.315351e+12,1.171868e+12,1.078491e+12,1.158913e+12,1.222408e+12,1.269012e+12,1.090515e+12,1.272839e+12
2,United States,USA,5.433000e+11,5.633000e+11,6.051000e+11,6.386000e+11,6.858000e+11,7.437000e+11,8.150000e+11,8.617000e+11,...,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13,1.869511e+13,1.947734e+13,2.053306e+13,2.138098e+13,2.106047e+13,2.331508e+13


In [78]:
# Hay muchas columnas, podemos aplicar un pandas.melt para solucionarlo
df_gdp = df_gdp.melt(["Country Name","Country Code"],value_name="gdp(current US$)")
# Observamos 
df_gdp

,Country Name,Country Code,2,gdp(current US$)
0,Japan,JPN,1960.0,4.430734e+10
1,Mexico,MEX,1960.0,1.304000e+10
2,United States,USA,1960.0,5.433000e+11
3,Japan,JPN,1961.0,5.350862e+10
4,Mexico,MEX,1961.0,1.416000e+10
...,...,...,...,...
181,Mexico,MEX,2020.0,1.090515e+12
182,United States,USA,2020.0,2.106047e+13
183,Japan,JPN,2021.0,4.940878e+12
184,Mexico,MEX,2021.0,1.272839e+12


In [79]:
# Normalizamos los nombres de las columnas
df_gdp.rename(columns={"Country Name" : "country", "Country Code" : "code", "Indicator Name" : "indicator", 2 : "year"},inplace=True)
# Agregamos el ID para cada país
df_gdp["idCountry"] = df_gdp["country"].replace({'United States': 1, 'Japan': 2, 'Mexico': 3})
# Observamos
df_gdp

,country,code,year,gdp(current US$),idCountry
0,Japan,JPN,1960.0,4.430734e+10,2
1,Mexico,MEX,1960.0,1.304000e+10,3
2,United States,USA,1960.0,5.433000e+11,1
3,Japan,JPN,1961.0,5.350862e+10,2
4,Mexico,MEX,1961.0,1.416000e+10,3
...,...,...,...,...,...
181,Mexico,MEX,2020.0,1.090515e+12,3
182,United States,USA,2020.0,2.106047e+13,1
183,Japan,JPN,2021.0,4.940878e+12,2
184,Mexico,MEX,2021.0,1.272839e+12,3


In [80]:
# Borramos la columna country, para garantizar una forma normal
df_gdp.pop("country")
# Reordenamos
df_gdp = df_gdp[["idCountry","code","year","gdp(current US$)"]]
# Observamos
df_gdp

,idCountry,code,year,gdp(current US$)
0,2,JPN,1960.0,4.430734e+10
1,3,MEX,1960.0,1.304000e+10
2,1,USA,1960.0,5.433000e+11
3,2,JPN,1961.0,5.350862e+10
4,3,MEX,1961.0,1.416000e+10
...,...,...,...,...
181,3,MEX,2020.0,1.090515e+12
182,1,USA,2020.0,2.106047e+13
183,2,JPN,2021.0,4.940878e+12
184,3,MEX,2021.0,1.272839e+12


In [81]:
df_gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   idCountry         186 non-null    int64  
 1   code              186 non-null    object 
 2   year              186 non-null    object 
 3   gdp(current US$)  186 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 5.9+ KB


In [82]:
# Colocamos el tipo de dato correcto
df_gdp['idCountry'] = df_gdp['idCountry'].astype('int64')
df_gdp['year'] = df_gdp['year'].astype('int64')
df_gdp['gdp(current US$)'] = df_gdp['gdp(current US$)'].astype('float64')
# Observamos
df_gdp

,idCountry,code,year,gdp(current US$)
0,2,JPN,1960,4.430734e+10
1,3,MEX,1960,1.304000e+10
2,1,USA,1960,5.433000e+11
3,2,JPN,1961,5.350862e+10
4,3,MEX,1961,1.416000e+10
...,...,...,...,...
181,3,MEX,2020,1.090515e+12
182,1,USA,2020,2.106047e+13
183,2,JPN,2021,4.940878e+12
184,3,MEX,2021,1.272839e+12


In [83]:
# Exportamos a formato JSON para su posterior análisis
df_gdp.to_json("Datasets/KPI 5/GDP (1960 - 2021).json")

---

# *Exportar a CSV*

#### Con el fin de facilitar el trabajo en la nube, se procede a exportar todos los archivos trabajados a formato CSV en la carpeta 'Datasets_csv'.

In [ ]:
# Exportamos a "Datasets_csv"

# Tabla general de paises (terremotos)
df_global.to_parquet("Datasets_csv/terremotos_global.parquet",compression="snappy")

# Tabla de paises con ID
df_country.to_csv("Datasets_csv/paises_id.csv",index=False)

# Paises objetivo (terremotos)
df_usa1.to_csv("Datasets_csv/Usa_terremotos.csv",index=False)
df_japon.to_csv("Datasets_csv/Japon_terremotos.csv",index=False)
df_mexico.to_csv("Datasets_csv/Mexico_terremotos.csv",index=False)

# KPI 1 
df_polizas.to_csv("Datasets_csv/polizas_japon_(kpi1).csv",index=False)
polizas_vigencia.to_csv("Datasets_csv/polizas_vigencia(kpi1).csv",index=False)
datos_hoja11.to_csv("Datasets_csv/nuevas polizas(kpi1).csv")
datos_hoja22.to_csv("Datasets_csv/monto asegurado(kpi1).csv")
datos_hoja33.to_csv("Datasets_csv/seguro contra terremotos(kpi1).csv")
datos_hoja44.to_csv("Datasets_csv/descuentos seguros(kpi1).csv")
datos_hoja55.to_csv("Datasets_csv/tasa penetración hogares(kpi1).csv")
seg_mex.to_parquet("Datasets_csv/seguros_mex_(kpi1).parquet")

# KPI 2
df_fatalidades.to_csv("Datasets_csv/Fatalidades_(kpi2).csv",index=False)

# KPI 3
df_participantes_simulacros.to_csv("Datasets_csv/Participantes_simulacros_paises_(kpi3).csv",index=False)
df_simulacros.to_csv("Datasets_csv/Participantes_simulacros_categorias_(kpi3).csv",index=False)

# KPI 4
datos_hoja1.to_csv("Datasets_csv/seguros_vida_japon1_(kpi4).csv",index=False)
datos_hoja2.to_csv("Datasets_csv/seguros_vida_japon2_(kpi4).csv",index=False)
datos_hoja3.to_csv("Datasets_csv/seguros_vida_japon3_(kpi4).csv",index=False)
datos_hoja4.to_csv("Datasets_csv/seguros_vida_japon4_(kpi4).csv",index=False)
datos_hoja5.to_csv("Datasets_csv/seguros_vida_japon5_(kpi4).csv",index=False)
datos_hoja6.to_csv("Datasets_csv/seguros_vida_japon6_(kpi4).csv",index=False)
datos_hoja7.to_csv("Datasets_csv/seguros_vida_japon7_(kpi4).csv",index=False)

# KPI 5
df_gdp.to_csv("Datasets_csv/GDP (1960 - 2021)_(kpi5).csv",index=False)

# `Futuras implementaciones al ETL`

+ Crear función que estandarice todas las columnas de las tablas
+ Crear más dimensiones para la estructura relacionel, por ejemplo "Estados"